In [1]:
using Random, Distributions

function xavier_init(input_dim::Int, output_dim::Union{Int,Nothing}=nothing)
    if output_dim === nothing
    scale = sqrt(2.0 / (input_dim + 1))
        return rand(Normal(0, scale), input_dim)
    end
    scale = sqrt(2.0 / (input_dim + output_dim))
    return rand(Normal(0, scale), input_dim, output_dim)
end

abstract type Node end

mutable struct InputNode <: Node
    output::AbstractVecOrMat
    name::String

    InputNode(output::AbstractVecOrMat; name="?"::String) = new(output, name)
    InputNode(output_size; name="?"::String) = new(zeros(output_size...), name)
end

struct ConstantNode <: Node
    output::AbstractVecOrMat
    name::String
    ConstantNode(output; name="?"::String) = new([output], name)
end

mutable struct VariableNode <: Node
    output::AbstractVecOrMat
    gradient::AbstractVecOrMat
    name::String

    VariableNode(output_size::Tuple{Int,Int}; name="?"::String) = new(xavier_init(output_size...), zeros(output_size...), name)
    VariableNode(output::AbstractVecOrMat; name="?"::String) = new(output, zeros(size(output)), name)
end

mutable struct OperationNode{F} <: Node
    inputs::Vector{Node}
    output::Union{AbstractVecOrMat,Nothing}
    gradient::Union{AbstractVecOrMat,Nothing}
    name::String
    OperationNode(fun::F, inputs::Vector{Node}; name="?"::String) where {F} =
        new{F}(inputs, nothing, nothing, name)

end

# Traversing

In [2]:


function visit(node::Node, visited::Set{Node}, order::Vector{Node})
    if node ∈ visited
    else
        push!(visited, node)
        push!(order, node)
    end
    return nothing
end
    
function visit(node::OperationNode, visited::Set{Node}, order::Vector{Node})
    if node ∈ visited
    else
        push!(visited, node)
        for input in node.inputs
            visit(input, visited, order)
        end
        push!(order, node)
    end
    return nothing
end

function topological_sort(head::Node)
    visited = Set{Node}()
    order = Vector{Node}()
    visit(head, visited, order)
    return order
end


function init_nodes!(order::Vector{Node})
    for node in order
        init_node!(node)
    end
end


init_node!(node::ConstantNode) = nothing
init_node!(node::InputNode) = nothing
init_node!(node::VariableNode) = nothing
function init_node!(node::OperationNode)
    println("init_node! ", typeof(node))
    println("input sizes: ", [(input.name,size(input.output)) for input in node.inputs])
    println("number of inputs: ", length(node.inputs))
    # println("inputs: ", [input.output for input in node.inputs])
    output_size = size(forward(node, [input.output for input in node.inputs]...))
    node.output = zeros(output_size)
    node.gradient = zeros(output_size)
    println("init_node-successful! ", typeof(node))
end

init_node! (generic function with 4 methods)

# Forward / Backward

In [3]:


reset!(node::ConstantNode) = nothing
reset!(node::InputNode) = nothing
reset!(node::VariableNode) = fill!(node.gradient, zero(eltype(node.gradient)))
reset!(node::OperationNode) = fill!(node.gradient, zero(eltype(node.gradient)))
function reset!(order::Vector{Node})
    for node in order
        reset!(node)
    end
    return nothing
end


reset_operations!(node::ConstantNode) = nothing
reset_operations!(node::InputNode) = nothing
reset_operations!(node::VariableNode) = nothing
reset_operations!(node::OperationNode) = fill!(node.gradient, zero(eltype(node.gradient)))


compute!(node::ConstantNode) = nothing
compute!(node::InputNode) = nothing
compute!(node::VariableNode) = nothing
function compute!(node::OperationNode)
    node.output = forward(node, [input.output for input in node.inputs]...)
end

function forward!(order::Vector{Node})
    for node in order
        compute!(node)
        reset_operations!(node)
    end
    return last(order).output
end


update!(node::ConstantNode, gradient) = nothing
update!(node::VariableNode, gradient) = let 
    if  isnothing(node.gradient)
        node.gradient = gradient
    else
        node.gradient .+= gradient
    end
end
update!(node::InputNode, gradient) = nothing
update!(node::OperationNode, gradient) =
    let
        if  isnothing(node.gradient)
            node.gradient = gradient
        else
            node.gradient .+= gradient
        end
    end

function backward!(order::Vector{Node}; seed=1.0)
    result = last(order)
    result.gradient = [seed]
    for node in reverse(order)
        backward!(node)
    end
    return nothing
end

function backward!(node::ConstantNode) end
function backward!(node::VariableNode) end
function backward!(node::InputNode) end
function backward!(node::OperationNode)
    inputs = node.inputs
    gradients = backward(node, [input.output for input in inputs]..., node.gradient)
    for (input, gradient) in zip(inputs, gradients)
        update!(input, gradient)
    end
    return nothing
end

adjust! (generic function with 1 method)

# Operators

In [4]:
import Base: +
+(x::Node, y::Node) = OperationNode(+, Node[x, y])
forward(::OperationNode{typeof(+)}, x, y) = return x .+ y
backward(::OperationNode{typeof(+)}, x, y, g) = tuple(g, g)


import Base: -
Base.Broadcast.broadcasted(-, x::Node, y::Node) = OperationNode(-, Node[x, y])
forward(::OperationNode{typeof(-)}, x, y) = return x .- y
backward(::OperationNode{typeof(-)}, x, y, g) = tuple(g, -g)


import Base: *
import LinearAlgebra: mul!
# x * y (aka matrix multiplication)
*(A::Node, x::Node) = OperationNode(mul!, Node[A, x])
forward(::OperationNode{typeof(mul!)}, A, x) = return A * x
backward(::OperationNode{typeof(mul!)}, A, x, g) = tuple(g * x', A' * g)


# x .* y (element-wise multiplication)
import Base: broadcast
broadcasted(*, x::Node, y::Node) = OperationNode(*, Node[x, y])
forward(::OperationNode{typeof(*)}, x, y) = return x .* y
# backward(::OperationNode{typeof(*)}, x, y, g) = tuple(g .* y, g .* x)
# backward(::OperationNode{typeof(*)}, x, y, g) = tuple(g .* y, x .* g)
backward(node::OperationNode{typeof(*)}, x, y, g) =
    let
return tuple(g .* y, g .* x)
    end

import Base: sum
sum(x::Node) = OperationNode(sum, Node[x])
forward(::OperationNode{typeof(sum)}, x) = return [sum(x)]
# backward(::OperationNode{typeof(sum)}, x, g) = tuple(g .* ones(size(x)))
# JEBIE SIE NA BACKWARDZIE
backward(::OperationNode{typeof(sum)}, x, g) =
    let
        𝟏 = ones(length(x))
        J = 𝟏'
        tuple(J' * g)
    end


import Base: ^
^(x::Node, n::Node) = OperationNode(^, Node[x, n])
forward(::OperationNode{typeof(^)}, x, n) = return x .^ n
backward(::OperationNode{typeof(^)}, x, n, g) =
    let
        return tuple(g .* n .* x .^ (n .- 1), g .* log.(abs.(x)) .* x .^ n)
    end


# tanh function overload with forward and backward methods
import Base: tanh
tanh(x::Node) = OperationNode(tanh, Node[x])
forward(::OperationNode{typeof(tanh)}, x) = return tanh.(x)
backward(::OperationNode{typeof(tanh)}, x, g) = tuple(g .* (1 .- tanh.(x) .^ 2))

# sigmoid function overload with forward and backward methods
import Base: broadcast
sigmoid(x::Node) = OperationNode(sigmoid, Node[x])
forward(::OperationNode{typeof(sigmoid)}, x) = return sigmoid.(x)
backward(::OperationNode{typeof(sigmoid)}, x, g) = tuple(g .* sigmoid.(x) .* (1 .- sigmoid.(x)))


cross_entropy_loss(y_hat::Node, y::Node) = OperationNode(cross_entropy_loss, Node[y_hat, y])
forward(::OperationNode{typeof(cross_entropy_loss)}, y_hat, y) =
    let
        y_hat = y_hat .- maximum(y_hat)
        y_hat = exp.(y_hat) ./ sum(exp.(y_hat))
        loss = sum(log.(y_hat) .* y) * -1.0
        return [loss]
    end
backward(::OperationNode{typeof(cross_entropy_loss)}, y_hat, y, g) =
    let
        y_hat = y_hat .- maximum(y_hat)
        y_hat = exp.(y_hat) ./ sum(exp.(y_hat))
        return tuple(g .* (y_hat - y))
    end



backward (generic function with 9 methods)

In [5]:

mutable struct Network
   inputs::Vector{InputNode}
   
   Wx::VariableNode
   Wh::VariableNode
   b::VariableNode
   h::VariableNode
   
   Wy::VariableNode
   by::VariableNode

   desired_output::InputNode

   output_graph::Vector{Node}
   loss_graph::Vector{Node}
    
end



function declare_RNN(input_lenght::Int, output_length::Int, neurons::Int)
   inputs = Vector{InputNode}()

      # Wx = VariableNode((input_lenght,neurons))
      Wx = VariableNode((neurons, input_lenght),name="Wx")
      Wh = VariableNode((neurons, neurons),name="Wh")
      b = VariableNode((neurons,1),name="b")
      h = VariableNode(zeros(neurons), name="h")
      Wy = VariableNode((output_length, neurons),name="Wy")
      by = VariableNode((output_length,1),name="by")
      desired_output = InputNode(zeros(output_length), name="desired_output")
      output_graph = Vector{Node}()
      loss_graph = Vector{Node}()
      Network(inputs, Wx, Wh, b, h, Wy, by, desired_output, output_graph, loss_graph)
   end

function unfold!(network::Network, n_sequences::Int)
      h = network.h
      Wh = network.Wh
      Wx = network.Wx
      b = network.b
      Wy = network.Wy
      by = network.by
      y = network.desired_output
      output_graph = network.output_graph
      loss_graph = network.loss_graph


      for i in 1:n_sequences
         x = InputNode(zeros(size(Wx.output)[2]), name="x")
         push!(network.inputs, x)
         h = tanh((Wx * x) .+ (Wh * h) .+ b)
      end
      y_hat = (Wy * h) .+ by

      network.output_graph= topological_sort(y_hat)
      network.loss_graph = topological_sort(cross_entropy_loss(y_hat, y))
      init_nodes!(network.loss_graph)
end

function feed_with_sequence!(network::Network, sequences...)
      for (input, x_i) in zip(network.inputs, sequences)
         input.output = x_i
      end
      return nothing
end

function feed_desired_output!(network::Network, y::AbstractVecOrMat)
      network.desired_output.output = y
      return nothing
end

function adjust!(net::Network, lr, batchsize)
   net.Wx.output .-= lr .* net.Wx.gradient ./ batchsize
   net.Wh.output .-= lr .* net.Wh.gradient ./ batchsize
   net.b.output .-= lr .* net.b.gradient ./ batchsize
   net.Wy.output .-= lr .* net.Wy.gradient ./ batchsize
   net.by.output .-= lr .* net.by.gradient ./ batchsize
   
end

adjust! (generic function with 2 methods)

In [11]:
function loss_and_accuracy(net::Network, data)
    correct = 0
    
    for (X_batch, Y_batch) in loader(test_data, batchsize=length(test_data))
    
        for i in 1:length(test_data)
            feed_with_sequence!(net,
                X_batch[1:196, i:i],
                X_batch[197:392, i:i],
                X_batch[393:588, i:i],
                X_batch[589:end, i:i]
            )
    
            y = @views Y_batch[:,i]
            ŷ = forward!(net.output_graph)
    
            if Flux.onecold(ŷ) == [Flux.onecold(y)]
                correct +=1
            end
        end
    end
    
    println("Correct: ", round(100 * correct/length(test_data); digits=2), "%")
end

loss_and_accuracy (generic function with 1 method)

In [7]:
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

loader (generic function with 1 method)

In [8]:
n_sequences = 4

net = declare_RNN(14*14, 10, 64)
unfold!(net, n_sequences)

init_node! OperationNode{typeof(mul!)}
input sizes: Tuple{String, Tuple{Int64, Vararg{Int64}}}[("Wx", (64, 196)), ("x", (196,))]
number of inputs: 2
init_node-successful! OperationNode{typeof(mul!)}
init_node! OperationNode{typeof(mul!)}
input sizes: Tuple{String, Tuple{Int64, Vararg{Int64}}}[("Wx", (64, 196)), ("x", (196,))]
number of inputs: 2
init_node-successful! OperationNode{typeof(mul!)}
init_node! OperationNode{typeof(mul!)}
input sizes: Tuple{String, Tuple{Int64, Vararg{Int64}}}[("Wx", (64, 196)), ("x", (196,))]
number of inputs: 2
init_node-successful! OperationNode{typeof(mul!)}
init_node! OperationNode{typeof(mul!)}
input sizes: Tuple{String, Tuple{Int64, Vararg{Int64}}}[("Wx", (64, 196)), ("x", (196,))]
number of inputs: 2
init_node-successful! OperationNode{typeof(mul!)}
init_node! OperationNode{typeof(mul!)}
input sizes: Tuple{String, Tuple{Int64, Vararg{Int64}}}[("Wh", (64, 64)), ("h", (64,))]
number of inputs: 2
init_node-successful! OperationNode{typeof(mul!)}
init_no

In [9]:



settings = (;
    epochs = 5,
    batchsize = 100,
    lr = 0.05
)




(epochs = 5, batchsize = 100, lr = 0.05)

In [12]:
loss_in_epoch = 0.0
losses = []

graph = net.loss_graph

@time for epoch in 1:settings.epochs
    println("Epoch: ", epoch)
    reset!(graph)
    loss_in_epoch = 0.0
    @time for (x, y) in loader(train_data, batchsize=settings.batchsize)
        reset!(graph)
        for i in 1:settings.batchsize
            feed_with_sequence!(net,
                view(x, 1:196,i),
                view(x, 197:392,i),
                view(x, 393:588,i),
                view(x, 589:784,i))
            feed_desired_output!(net, view(y, :, i))
            forward!(graph)
            loss = forward!(graph)
            loss_in_epoch += first(loss)
            backward!(graph)
        end
        adjust!(net, settings.lr, settings.batchsize)
    end
    println("Current loss: ", loss_in_epoch)
    push!(losses, first(loss_in_epoch))
    loss_and_accuracy(net, train_data)

end

Epoch: 1
 30.599268 seconds (41.73 M allocations: 34.225 GiB, 5.23% gc time)
Current loss: 17310.409321171443
Correct: 93.07%
Epoch: 2
 30.595451 seconds (41.73 M allocations: 34.225 GiB, 5.45% gc time)
Current loss: 13776.991677565367
Correct: 94.09%
Epoch: 3
 30.138373 seconds (41.73 M allocations: 34.225 GiB, 5.41% gc time)
Current loss: 11716.25195472323
Correct: 94.82%
Epoch: 4
 28.750153 seconds (41.73 M allocations: 34.225 GiB, 5.34% gc time)
Current loss: 10259.146006783369
Correct: 95.16%
Epoch: 5
 28.798814 seconds (41.73 M allocations: 34.225 GiB, 5.42% gc time)
Current loss: 9192.107041483754
Correct: 95.54%
155.207963 seconds (217.15 M allocations: 172.522 GiB, 5.22% gc time, 0.33% compilation time)


In [ ]:
# loss_in_epoch = 0.0
# losses = []

# graph = net.loss_graph

# @time for epoch in 1:settings.epochs
#     loss_in_epoch = 0.0
#     println("Epoch: ", epoch)
#     reset!(graph)
#     @time for (x, y) in loader(train_data, batchsize=settings.batchsize)
#         feed_with_sequence!(net,
#             x[1:196, :],
#             x[197:392, :],
#             x[393:588, :],
#             x[589:784, :]
#         )
#         feed_desired_output!(net, y)
#         currentloss = forward!(graph)
#         backward!(graph)
#         loss_in_epoch += first(currentloss)
#         end
#     # foreach(x -> x.gradient ./= (length(train_data)/settings.batchsize), net.variables)
# adjust!(net, settings.lr, length(train_data))
#     # @show net.variables[1].gradient
#     println("Current loss: ", loss_in_epoch)
#     push!(losses, first(loss_in_epoch))
#     println("Accuracy: ", loss_and_accuracy(net, train_data))
# end

In [ ]:
i=0
@time for (x, y) in loader(train_data, batchsize=settings.batchsize)
i+=1
end
@show i
length(train_data)

  0.106907 seconds (11.02 k allocations: 181.709 MiB, 5.28% gc time)
i = 600


60000

In [ ]:
function loss_and_accuracy(net::Network, data)
    correct = 0
    graph = net.output_graph

    for(x,y) in loader(data; batchsize=length(data))
        for i in 1:length(x)
            feed_with_sequence!(net,
            view(x, 1:196,i),
            view(x, 197:392,i),
            view(x, 393:588,i),
            view(x, 589:784,i)
            )

            y = y[:,i]
            ŷ = forward!(graph)
    
            if Flux.onecold(ŷ) == [Flux.onecold(y)]
                correct +=1
            end

        end
    end
    println("Correct: ", round(100 * correct/length(data); digits=2), "%")
end

loss_and_accuracy(net, train_data)

Correct: 91.72%
